In [1]:
import pandas as pd

In [2]:
def calculate_final_driver_places(conn):

    points_query = '''
        SELECT r.year, ds.driverId, d.forename || ' ' || d.surname AS driver_name, ds.points
        FROM driver_standings ds
        INNER JOIN (
            SELECT r1.year, MAX(r1.date) AS last_race_date
            FROM races r1
            GROUP BY r1.year
        ) last_race ON ds.raceId = (
            SELECT r2.raceId
            FROM races r2
            WHERE r2.year = last_race.year AND r2.date = last_race.last_race_date
            LIMIT 1
        )
        INNER JOIN races r ON ds.raceId = r.raceId AND r.date = last_race.last_race_date
        INNER JOIN drivers d ON ds.driverId = d.driverId;
    '''

    points_df = pd.read_sql_query(points_query, conn)

    points_df = points_df.sort_values(by=['year', 'points'], ascending=[True, False])

    points_df['place'] = points_df.groupby('year')['points'].rank(ascending=False, method='min').astype(int)

    return points_df


In [3]:
def calculate_final_constructor_places(conn):
    
    constructor_points_query = '''
        WITH LastRacePerSeason AS (
        SELECT
            r.year,
            MAX(r.date) AS last_race_date
        FROM
            races r
        WHERE
            r.year BETWEEN 1950 AND 2022
        GROUP BY
            r.year
        )

        SELECT
            lrs.year,
            cs.constructorId,
            c.name AS constructor_name,
            cs.points,
            RANK() OVER (PARTITION BY lrs.year ORDER BY cs.points DESC) AS place
        FROM
            LastRacePerSeason lrs
        JOIN
            races r ON lrs.year = r.year AND lrs.last_race_date = r.date
        JOIN
            constructor_standings cs ON r.raceId = cs.raceId
        JOIN
            constructors c ON cs.constructorId = c.constructorId
        ORDER BY
            lrs.year DESC, cs.points DESC;
    '''

    constructor_points_df = pd.read_sql_query(constructor_points_query, conn)

    return constructor_points_df

In [4]:
def get_max_min_points_for_season(df):
    max_min_points_df = df.groupby('year').agg(
        max_points=('points', 'max'),
        min_points=('points', 'min')
    ).reset_index()

    return max_min_points_df

In [5]:
def get_number_of_drivers_for_season(df):
    num_drivers_df = df.groupby('year')['driverId'].nunique().reset_index(name='num_drivers')
    return num_drivers_df